In [1]:
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
import numpy as np
import pandas as pd

# Load dataset

In [2]:
artists, users, artist_user_plays = get_lastfm()

# Train model

In [3]:
# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
artist_user_plays = bm25_weight(artist_user_plays, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_plays = artist_user_plays.T.tocsr()

In [4]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(user_plays)

/home/egor/projects/rec-sys-playground/last-fm-als/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

# Get recommandations

In [12]:
# Get recommendations for the a single user
def get_recommendations(model, user_plays, user_id, filter_already_liked_items):
    ids, scores = model.recommend(user_id, user_plays[user_id], N=10, filter_already_liked_items=filter_already_liked_items)
    return pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.in1d(ids, user_plays[user_id].indices)})

In [13]:
get_recommendations(model=model, user_plays=user_plays, user_id=12345, filter_already_liked_items=False)

,artist,score,already_liked
0,puissance,1.010351,True
1,laibach,0.987085,False
2,rome,0.982097,True
3,von thronstahl,0.974682,True
4,the coffinshakers,0.972303,True
5,mortiis,0.970911,True
6,d-a-d,0.957738,True
7,kreuzweg ost,0.956606,True
8,arditi,0.952835,True
9,triarii,0.951422,True


In [14]:
get_recommendations(model=model, user_plays=user_plays, user_id=12345, filter_already_liked_items=True)

,artist,score,already_liked
0,laibach,0.987085,False
1,spiritual front,0.943995,False
2,ordo rosarius equilibrio,0.941346,False
3,forseti,0.928550,False
4,blood axis,0.916849,False
5,sonne hagal,0.912741,False
6,ordo equilibrio,0.909935,False
7,of the wand and the moon,0.908993,False
8,dernière volonté,0.904741,False
9,in slaughter natives,0.904502,False


# Recommend similar items

In [15]:
# get related items for the beatles (itemid = 25512)
ids, scores= model.similar_items(252512)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": artists[ids], "score": scores})

,artist,score
0,the beatles,1.000000
1,the beach boys,0.993613
2,the rolling stones,0.993535
3,john lennon,0.992937
4,bob dylan,0.992382
5,the who,0.992356
6,simon & garfunkel,0.991736
7,david bowie,0.991602
8,led zeppelin,0.991265
9,the white stripes,0.990274
